In [1]:
import pandas as pd
import sys
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from IPython.display import display, HTML
from scipy.stats import spearmanr
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import zipfile
import random
import gzip
import sys
import h5py
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", None)

display(HTML("""
<style>
.dataframe td, .dataframe th {
    white-space: nowrap !important;
}
</style>
"""))

In [2]:
import networkx as nx

In [3]:
# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import DATAPATH, CONFIGPATH

# Load pathogen info
# pathogen_code = sys.argv[1]
pathogen_code = 'mtuberculosis'
df = pd.read_csv(os.path.join(CONFIGPATH, 'pathogens.csv'))
row = df.loc[df["code"].eq(pathogen_code)]
if row.empty: 
    raise SystemExit(f"Unknown code: {pathogen_code}")
pathogen = row.iloc[0]["pathogen"]

# Create output directory
OUTPUT = os.path.join(root, "..", "output")

# Shared columns
KEYS = ["assay_id", "activity_type", "unit"]

# Columns to take from each table
COLUMNS_CLEANED = ["assay_id", "assay_type", "assay_organism", "doc_chembl_id", "target_type", "target_chembl_id", "target_organism", "activity_type", 
                "unit", "canonical_unit", "activities", "nan_values", "cpds", "direction", "activity_comment_counts", "standard_text_count"]
# COLUMNS_CLUSTERS = ['clusters_0.3', 'clusters_0.6', 'clusters_0.85']
COLUMNS_DATASETS = ["equal", 'higher', 'lower', "target_type_curated_extra", "dataset_type", "cpds_qt", "min_", "p1", "p25", "p50", "p75", "p99", "max_", "pos_ql", "ratio_ql", "cpds_ql"]

In [4]:
# Load assays info
ASSAYS_CLEANED = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_cleaned.csv"))
# ASSAYS_CLUSTERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_clusters.csv"))
ASSAYS_PARAMETERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_parameters.csv"))
ASSAYS_DATASETS_ = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_datasets.csv"))
INDIVIDUAL_LM = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "individual_LM.csv"))

# Get assay to quantitative data info
assay_to_qt_info = defaultdict(list)
for assay_id, activity_type, unit, expert_cutoff, ratio_qt in ASSAYS_DATASETS_[['assay_id', 'activity_type', 'unit', 'expert_cutoff', 'ratio_qt']].values:
    assay_to_qt_info[tuple([assay_id, activity_type, unit])].append([expert_cutoff, ratio_qt])

# Unique row per assay
ASSAYS_DATASETS = ASSAYS_DATASETS_[KEYS + COLUMNS_DATASETS].drop_duplicates().reset_index(drop=True)

# Get cutoffs and ratios
cutoffs = [";".join([str(j[0]) for j in assay_to_qt_info[tuple(i)]]) for i in ASSAYS_DATASETS[['assay_id', 'activity_type', 'unit']].values]
ratios = [";".join([str(j[1]) for j in assay_to_qt_info[tuple(i)]]) for i in ASSAYS_DATASETS[['assay_id', 'activity_type', 'unit']].values]
cutoffs = [i if i != 'nan' else np.nan for i in cutoffs]
ratios = [i if i != 'nan' else np.nan for i in ratios]

# Store results
ASSAYS_DATASETS.insert(8, 'cutoffs', cutoffs)
ASSAYS_DATASETS.insert(9, 'ratios', ratios)

# Merge everything
ASSAYS_MASTER = ASSAYS_CLEANED.merge(ASSAYS_PARAMETERS,on=KEYS, how="left", validate="1:1")
ASSAYS_MASTER = ASSAYS_MASTER.merge(ASSAYS_DATASETS,on=KEYS, how="left", validate="1:1")

In [5]:
len(ASSAYS_CLEANED), len(ASSAYS_PARAMETERS), len(ASSAYS_DATASETS)

(10532, 10532, 10532)

In [6]:
ALL_COLS = ["assay_id", "assay_type", "assay_organism", "target_organism", "organism_curated", "doc_chembl_id", "target_type", "target_type_curated", "target_type_curated_extra", 
          "target_chembl_id", "target_chembl_id_curated", "target_name_curated", "bao_label", "source_label", "strain", "atcc_id", "mutations", "known_drug_resistances", "media",
          "activity_type", "unit", "activities", "nan_values", "cpds", "frac_cs", "direction", "act_flag", 'inact_flag', "equal", "higher", "lower", "dataset_type", "cutoffs", "ratios", 
          "cpds_qt", "pos_ql", "ratio_ql", "cpds_ql", "min_", "p1", "p25", "p50", "p75", "p99", "max_", 'Accepted', 'Considered']

In [7]:
def get_all_results_from_individual_modeling(LABELS):
    RESULTS, CONSIDERED_ASSAYS = {}, {}
    for LABEL in LABELS:
        RESULTS[LABEL] = {}
        CONSIDERED_ASSAYS[LABEL] = set()
        rows = INDIVIDUAL_LM[INDIVIDUAL_LM[LABEL]][["assay_id", "activity_type", "unit", "expert_cutoff", f"{LABEL}_AVG"]].values
        for assay_id, activity_type, unit, expert_cutoff, auroc in rows:
            key = (assay_id, activity_type, unit)
            CONSIDERED_ASSAYS[LABEL].add(key)
            if auroc > 0.7:
                if key not in RESULTS[LABEL]:
                    RESULTS[LABEL][key] = [expert_cutoff, auroc]
                elif auroc > RESULTS[LABEL][key][1]:
                    RESULTS[LABEL][key] = [expert_cutoff, auroc]
    return RESULTS, CONSIDERED_ASSAYS

def where_considered(key, LABELS, CONSIDERED_ASSAYS):
    considered = []
    for LABEL in LABELS:
        if key in CONSIDERED_ASSAYS[LABEL]:
            considered.append(LABEL)
    if len(considered) > 0:
        return ";".join(considered)
    else:
        return np.nan
    
def where_accepted(key, LABELS, ACCEPTED_ASSAYS):
    accepted = []
    for LABEL in LABELS:
        if key in ACCEPTED_ASSAYS[LABEL]:
            accepted.append(LABEL)
    if len(accepted) > 0:
        return ";".join(accepted)
    else:
        return np.nan

# Get results from individual modeling ABCD
LABELS = ['A', 'B', 'C', 'D']
ACCEPTED_ASSAYS, CONSIDERED_ASSAYS = get_all_results_from_individual_modeling(LABELS)

col_accepted, col_considered = [], []
for assay_id, activity_type, unit in ASSAYS_MASTER[["assay_id", "activity_type", "unit"]].values:
    # Get strategies in which this assay is considered and accepted
    key = tuple([assay_id, activity_type, unit])
    col_considered.append(where_considered(key, LABELS, CONSIDERED_ASSAYS))
    col_accepted.append(where_accepted(key, LABELS, ACCEPTED_ASSAYS))
ASSAYS_MASTER['Accepted'] = col_accepted
ASSAYS_MASTER['Considered'] = col_considered

In [8]:
ASSAYS_MASTER = ASSAYS_MASTER[ALL_COLS]

In [9]:
ASSAYS_MASTER[(ASSAYS_MASTER['Accepted'].isna()) & (ASSAYS_MASTER['target_type_curated_extra'] == 'ORGANISM')][:10]

,assay_id,assay_type,assay_organism,target_organism,organism_curated,doc_chembl_id,target_type,target_type_curated,target_type_curated_extra,target_chembl_id,target_chembl_id_curated,target_name_curated,bao_label,source_label,strain,atcc_id,mutations,known_drug_resistances,media,activity_type,unit,activities,nan_values,cpds,frac_cs,direction,act_flag,inact_flag,equal,higher,lower,dataset_type,cutoffs,ratios,cpds_qt,pos_ql,ratio_ql,cpds_ql,min_,p1,p25,p50,p75,p99,max_,Accepted,Considered
27,CHEMBL2094261,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL2094260,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_MBOX,NaN,NaN,NaN,NaN,NaN,INHIBITION,%,399,0,399,0.00288,1.0,0,0,399,0,0,quantitative,25.0;50.0;75.0,0.128;0.028;0.018,399.0,NaN,NaN,NaN,-14.000,-6.706,6.810,12.000,18.850,99.804,102.000,NaN,NaN
28,CHEMBL2094262,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL2094260,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_MBOX,NaN,NaN,NaN,NaN,NaN,INHIBITION,%,399,0,399,0.00288,1.0,0,0,399,0,0,quantitative,25.0;50.0;75.0,0.286;0.165;0.113,399.0,NaN,NaN,NaN,-15.400,-4.780,7.625,14.800,31.250,98.812,101.000,NaN,NaN
34,CHEMBL4649971,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3988442,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,GATES_LIBRARY,NaN,NaN,NaN,NaN,NaN,IC50,umol.L-1,269,0,200,0.00144,-1.0,0,0,165,104,0,quantitative,5.0;10.0;20.0,0.055;0.12;0.23,200.0,NaN,NaN,NaN,1.315,1.757,21.996,49.864,100.000,100.000,100.000,NaN,NaN
38,CHEMBL5345966,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5344439,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,H37Rv,NaN,NaN,NaN,Sulas semisynthetic medium,MIC99,umol.L-1,172,0,172,0.00124,-1.0,0,0,135,36,1,quantitative,5.0;10.0;20.0,0.25;0.297;0.39,172.0,NaN,NaN,NaN,0.030,0.106,7.000,64.000,250.000,250.000,250.000,NaN,NaN
39,CHEMBL5345967,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5344439,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,H37Rv,NaN,NaN,NaN,Sulas semisynthetic medium,MIC99,umol.L-1,172,0,172,0.00124,-1.0,0,0,134,37,1,quantitative,5.0;10.0;20.0,0.221;0.262;0.355,172.0,NaN,NaN,NaN,0.030,0.106,8.000,125.000,250.000,250.000,250.000,NaN,NaN
42,CHEMBL4388638,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL4387722,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,H37Rv,NaN,NaN,NaN,NaN,INHIBITION,%,146,0,146,0.00105,1.0,0,0,146,0,0,quantitative,25.0;50.0;75.0,0.493;0.336;0.151,146.0,NaN,NaN,NaN,0.000,0.000,0.000,23.000,60.500,92.650,95.000,NaN,NaN
43,CHEMBL2032580,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL2029208,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,H37Ra,NaN,NaN,NaN,NaN,MIC,umol.L-1,136,0,136,0.00098,-1.0,0,0,95,41,0,quantitative,5.0;10.0;20.0,0.169;0.331;0.426,136.0,NaN,NaN,NaN,0.031,0.281,6.254,49.113,207.938,207.938,207.938,NaN,NaN
49,CHEMBL747348,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL1122718,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,NaN,NaN,NaN,NaN,NaN,MIC,umol.L-1,118,0,118,0.00085,-1.0,0,0,58,60,0,quantitative,5.0;10.0;20.0,0.0;0.0;0.0,118.0,NaN,NaN,NaN,32.121,55.683,302.589,1866.134,1866.134,1866.134,1866.134,NaN,NaN
50,CHEMBL3832767,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3832761,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_PBOX,NaN,NaN,Na

In [10]:
ASSAYS_MASTER[ASSAYS_MASTER['Considered'].isna()][:20]

,assay_id,assay_type,assay_organism,target_organism,organism_curated,doc_chembl_id,target_type,target_type_curated,target_type_curated_extra,target_chembl_id,target_chembl_id_curated,target_name_curated,bao_label,source_label,strain,atcc_id,mutations,known_drug_resistances,media,activity_type,unit,activities,nan_values,cpds,frac_cs,direction,act_flag,inact_flag,equal,higher,lower,dataset_type,cutoffs,ratios,cpds_qt,pos_ql,ratio_ql,cpds_ql,min_,p1,p25,p50,p75,p99,max_,Accepted,Considered
8,CHEMBL4649947,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv),Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,SINGLE PROTEIN,SINGLE PROTEIN,CHEMBL4662921,NaN,Asp RNA synthetase-1,assay format,GATES_LIBRARY,H37Rv,ATCC 25618,NaN,NaN,NaN,PERCENTEFFECT,%,8841,0,8841,0.06381,1.0,0,0,8841,0,0,quantitative,25.0;50.0;75.0,0.003;0.001;0.0,8841.0,NaN,NaN,NaN,-13.910,-8.344,-2.622,-0.280,2.323,12.910,65.320,NaN,NaN
23,CHEMBL4649972,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv),Mycobacterium tuberculosis,CHEMBL3988442,PROTEIN COMPLEX,PROTEIN COMPLEX,SINGLE PROTEIN,CHEMBL4662931,NaN,ClpP1P2,assay format,GATES_LIBRARY,H37Rv,ATCC 25618,NaN,NaN,NaN,IC50,umol.L-1,674,0,674,0.00486,-1.0,0,0,7,667,0,quantitative,0.5;1.0;5.0,0.0;0.0;0.0,674.0,NaN,NaN,NaN,7.254,95.781,100.000,100.000,100.000,100.000,100.000,NaN,NaN
24,CHEMBL4649957,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv),Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,SINGLE PROTEIN,SINGLE PROTEIN,CHEMBL4662922,NaN,Mt-TrpRS,assay format,GATES_LIBRARY,H37Rv,ATCC 25618,NaN,NaN,NaN,IC50,umol.L-1,612,0,597,0.00431,-1.0,0,0,2,610,0,quantitative,0.5;1.0;5.0,0.0;0.0;0.0,597.0,NaN,NaN,NaN,31.401,194.984,194.984,194.984,194.984,194.984,194.984,NaN,NaN
26,CHEMBL4649941,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv),Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,SINGLE PROTEIN,SINGLE PROTEIN,CHEMBL4662928,NaN,MtCoaBC,assay format,GATES_LIBRARY,H37Rv,ATCC 25618,NaN,NaN,NaN,IC50,umol.L-1,914,0,439,0.00317,-1.0,0,0,9,905,0,quantitative,0.5;1.0;5.0,0.0;0.0;0.002,439.0,NaN,NaN,NaN,4.134,47.157,50.119,50.119,50.119,50.119,50.119,NaN,NaN
27,CHEMBL2094261,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL2094260,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_MBOX,NaN,NaN,NaN,NaN,NaN,INHIBITION,%,399,0,399,0.00288,1.0,0,0,399,0,0,quantitative,25.0;50.0;75.0,0.128;0.028;0.018,399.0,NaN,NaN,NaN,-14.000,-6.706,6.810,12.000,18.850,99.804,102.000,NaN,NaN
28,CHEMBL2094262,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL2094260,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_MBOX,NaN,NaN,NaN,NaN,NaN,INHIBITION,%,399,0,399,0.00288,1.0,0,0,399,0,0,quantitative,25.0;50.0;75.0,0.286;0.165;0.113,399.0,NaN,NaN,NaN,-15.400,-4.780,7.625,14.800,31.250,98.812,101.000,NaN,NaN
32,CHEMBL4649965,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv),Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,SINGLE PROTEIN,SINGLE PROTEIN,CHEMBL4105939,NaN,Polyketide synthase 13 Thioesterase (PKS13),assay format,GATES_LIBRARY,H37Rv,ATCC 25618,NaN,NaN,NaN,IC50,umol.L-1,601,0,302,0.00218,-1.0,0,0,255,346,0,quantitative,0.5;1.0;5.0,0.0;0.003;0.02,302.0,NaN,NaN,NaN,0.803,3.939,19.306,100.000,100.000,100.000,100.000,NaN,NaN
34,CHEMBL4649971,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3988442,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,GATES_LIBRARY,NaN,NaN,NaN,NaN,NaN,IC50,umol.L-1,269,0,200,0.00144,-1.0,0,0,165,104,0,quantitative,5.0;10.0;20.0,0.055;0.12;0.23,200.0,NaN,NaN,NaN,1.315,1.757,21.996,49.864,100.000,100.000,100.000,NaN,NaN
38,CHEMBL5345966,F,Mycobacterium tuberculosis,Mycobacterium tuber

In [11]:
cpds_by_pair = (ASSAYS_MASTER[(ASSAYS_MASTER['Considered'].isna()) & (ASSAYS_MASTER['target_type_curated_extra'] == 'ORGANISM')]
    .groupby(["activity_type", "unit"], dropna=False)["cpds"]
    .sum()
    .sort_values(ascending=False)
    .reset_index(name="cpds_sum")
)

total = cpds_by_pair["cpds_sum"].sum()
cpds_by_pair["cumulative_proportion"] = (cpds_by_pair["cpds_sum"].cumsum() / total)
cpds_by_pair[:5]

,activity_type,unit,cpds_sum,cumulative_proportion
0,MIC,umol.L-1,44664,0.573505
1,MIC90,umol.L-1,9418,0.694436
2,INHIBITION,%,5482,0.764827
3,MIC99,umol.L-1,4520,0.822866
4,GI,%,2971,0.861015


In [66]:
df = ASSAYS_MASTER[
    (ASSAYS_MASTER["Considered"].isna()) &
    (ASSAYS_MASTER["target_type_curated_extra"] == "ORGANISM") &
    (ASSAYS_MASTER["activity_type"] == "MIC") &
    (ASSAYS_MASTER["unit"] == "umol.L-1")
].copy()

In [67]:
df

,assay_id,assay_type,assay_organism,target_organism,organism_curated,doc_chembl_id,target_type,target_type_curated,target_type_curated_extra,target_chembl_id,target_chembl_id_curated,target_name_curated,bao_label,source_label,strain,atcc_id,mutations,known_drug_resistances,media,activity_type,unit,activities,nan_values,cpds,frac_cs,direction,act_flag,inact_flag,equal,higher,lower,dataset_type,cutoffs,ratios,cpds_qt,pos_ql,ratio_ql,cpds_ql,min_,p1,p25,p50,p75,p99,max_,Accepted,Considered
43,CHEMBL2032580,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL2029208,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,H37Ra,NaN,NaN,NaN,NaN,MIC,umol.L-1,136,0,136,0.00098,-1.0,0,0,95,41,0,quantitative,5.0;10.0;20.0,0.169;0.331;0.426,136.0,NaN,NaN,NaN,0.031,0.281,6.254,49.113,207.938,207.938,207.938,NaN,NaN
49,CHEMBL747348,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL1122718,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,NaN,NaN,NaN,NaN,NaN,MIC,umol.L-1,118,0,118,0.00085,-1.0,0,0,58,60,0,quantitative,5.0;10.0;20.0,0.0;0.0;0.0,118.0,NaN,NaN,NaN,32.121,55.683,302.589,1866.134,1866.134,1866.134,1866.134,NaN,NaN
50,CHEMBL3832767,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3832761,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_PBOX,NaN,NaN,NaN,NaN,7H9/ADC/Tw,MIC,umol.L-1,116,0,116,0.00084,-1.0,0,0,107,8,1,quantitative,5.0;10.0;20.0,0.336;0.483;0.707,116.0,NaN,NaN,NaN,0.020,0.020,2.300,12.500,25.000,50.000,50.000,NaN,NaN
54,CHEMBL3832765,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3832761,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_PBOX,NaN,NaN,NaN,NaN,7H9/ADC/Tw,MIC,umol.L-1,116,0,116,0.00084,-1.0,0,0,86,28,2,quantitative,5.0;10.0;20.0,0.319;0.362;0.509,116.0,NaN,NaN,NaN,0.020,0.025,3.130,19.000,50.000,50.000,50.000,NaN,NaN
55,CHEMBL3832766,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3832761,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,MMV_PBOX,NaN,NaN,NaN,NaN,7H9/ADC/Tw,MIC,umol.L-1,116,0,116,0.00084,-1.0,0,0,79,36,1,quantitative,5.0;10.0;20.0,0.259;0.362;0.414,116.0,NaN,NaN,NaN,0.020,0.028,4.700,25.000,50.000,50.000,50.000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,CHEMBL5229084,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5226349,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,NaN,NaN,NaN,NaN,NaN,MIC,umol.L-1,1,0,1,0.00001,-1.0,0,0,1,0,0,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,1.0,NaN,NaN,NaN,2.510,2.510,2.510,2.510,2.510,2.510,2.510,NaN,NaN
10504,CHEMBL5226839,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5226287,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,H37Rv,NaN,NaN,NaN,NaN,MIC,umol.L-1,1,0,1,0.00001,-1.0,0,0,1,0,0,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,1.0,NaN,NaN,NaN,0.102,0.102,0.102,0.102,0.102,0.102,0.102,NaN,NaN
10507,CHEMBL5226809,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5226287,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,NaN,NaN,NaN,NaN,NaN,MIC,umol.L-1,1,0,1,0.00001,-1.0,0,0,0,0,1,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,1.0,NaN,NaN,NaN,0.024,0.024,0.024,0.024,0.024,0.024,0.024,NaN,NaN
10515,CHEMBL5226819,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5226287,ORGANISM,ORGANISM

In [68]:
cols = ["strain", "atcc_id", "media", "mutations", "known_drug_resistances"]
df[cols].isna().mean().sort_values(ascending=False)

mutations                 0.913521
media                     0.892990
atcc_id                   0.838656
known_drug_resistances    0.736416
strain                    0.197014
dtype: float64

In [78]:
strain_weight = 0.5
atcc_id_weight = 0.2
media_weight = 0.1
mutations_weight = 0.1
known_drug_resistances_weight = 0.1

# Normalize fields
df["strain"]  = df["strain"].fillna("").astype(str).str.strip().str.lower()
df["atcc_id"] = df["atcc_id"].fillna("").astype(str).str.strip().str.upper()
df["media"] = df["media"].fillna("").astype(str).str.strip().str.upper()
df["mutations"] = df["mutations"].fillna("").astype(str).str.strip().str.upper()
df["known_drug_resistances"] = df["known_drug_resistances"].fillna("").astype(str).str.strip().str.upper()

# Map node to info
df["_node"] = list(zip(df["assay_id"], df["activity_type"], df["unit"]))
node_to_atcc = dict(zip(df["_node"], df["atcc_id"]))
node_to_media = dict(zip(df["_node"], df["media"]))
node_to_mutations = dict(zip(df["_node"], df["mutations"]))
node_to_known_drug_resistances = dict(zip(df["_node"], df["known_drug_resistances"]))

def to_set(s):
    s = "" if pd.isna(s) else str(s).strip().upper()
    if s == "":
        return set()
    return {x.strip() for x in s.split(";") if x.strip()}

node_to_mutations = {n: to_set(m) for n, m in zip(df["_node"], df["mutations"])}
node_to_known_drug_resistances = {n: to_set(m) for n, m in zip(df["_node"], df["known_drug_resistances"])}

G = nx.Graph()

# Add nodes (assay_id, activity_type, unit)
for r in df.itertuples(index=False):
    G.add_node((r.assay_id, r.activity_type, r.unit))

# Add edges: connect assays with the same strain
for strain, sub in df[df["strain"] != ""].groupby("strain"):

    # Unique nodes in this strain
    nodes = list({(r.assay_id, r.activity_type, r.unit) for r in sub.itertuples(index=False)})

    # Connect all pairs having the same strain
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            G.add_edge(nodes[i], nodes[j], weight_strain=strain_weight)

print("Nodes:", G.number_of_nodes(), "Same strain edges:", G.number_of_edges())

# ATCC ID
n_with_atcc_id = (df["atcc_id"] != "").sum()
print("Nodes with ATCC ID:", n_with_atcc_id, "from", len(df), f"({100*n_with_atcc_id/len(df):.2f}%)")
boosted_atcc = 0
for u, v, data in G.edges(data=True):
    a = node_to_atcc[u]
    b = node_to_atcc[v]
    if a != "" and b != "" and a == b:
        data["weight_atcc_id"] = atcc_id_weight
        boosted_atcc += 1
    elif (a == "" and b != "") or (a != "" and b == ""):
        data["weight_atcc_id"] = atcc_id_weight / 2
        boosted_atcc += 1
    else:
        data["weight_atcc_id"] = 0
print(f"ATCC-boosted edges: {boosted_atcc} from {G.number_of_edges()} ({100*round(boosted_atcc/G.number_of_edges(), 4)}%)")

# MEDIA
n_with_media = (df["media"] != "").sum()
print("Nodes with media:", n_with_media, "from", len(df), f"({100*n_with_media/len(df):.2f}%)")
boosted_media = 0
for u, v, data in G.edges(data=True):
    a = node_to_media[u]
    b = node_to_media[v]
    if a != "" and b != "" and a == b:
        data["weight_media"] = media_weight
        boosted_media += 1
    # elif (a == "" and b != "") or (a != "" and b == ""):
    #     data["weight_media"] = media_weight / 2
    #     boosted_media += 1
    else:
        data["weight_media"] = 0
print(f"MEDIA-boosted edges: {boosted_media} from {G.number_of_edges()} ({100*boosted_media/G.number_of_edges():.2f}%)")


boosted_mut = 0
for u, v, data in G.edges(data=True):
    a = node_to_mutations[u]
    b = node_to_mutations[v]

    if len(a) > 0 and len(b) > 0:
        sim = len(a & b) / len(a | b)
        data["weight_mutations"] = mutations_weight * sim
        boosted_mut += 1 if sim > 0 else 0
    # elif len(a) == 0 and len(b) == 0:
    #     data["weight_mutations"] = mutations_weight / 2
    #     boosted_mut += 1
    else:
        data["weight_mutations"] = 0

print(f"MUTATIONS-boosted edges: {boosted_mut} from {G.number_of_edges()} ({100*boosted_mut/G.number_of_edges():.2f}%)")

boosted_kdr = 0
for u, v, data in G.edges(data=True):
    a = node_to_known_drug_resistances[u]
    b = node_to_known_drug_resistances[v]

    if len(a) > 0 and len(b) > 0:
        sim = len(a & b) / len(a | b) 
        data["weight_known_drug_resistances"] = known_drug_resistances_weight * sim
        boosted_kdr += 1 if sim > 0 else 0
    else:
        data["weight_known_drug_resistances"] = 0.0

print(f"KDR-boosted edges: {boosted_kdr} from {G.number_of_edges()} ({100*boosted_kdr/G.number_of_edges():.2f}%)")

Nodes: 4822 Same strain edges: 2109778
Nodes with ATCC ID: 778 from 4822 (16.13%)
ATCC-boosted edges: 962424 from 2109778 (45.62%)
Nodes with media: 516 from 4822 (10.70%)
MEDIA-boosted edges: 1410 from 2109778 (0.07%)
MUTATIONS-boosted edges: 88 from 2109778 (0.00%)
KDR-boosted edges: 9264 from 2109778 (0.44%)


In [81]:
# compute final weight for every edge
for _, _, data in G.edges(data=True):
    data["weight"] = (
        data["weight_strain"] +
        data["weight_atcc_id"] +
        data["weight_media"] +
        data["weight_mutations"] +
        data["weight_known_drug_resistances"]
    )


In [104]:
THR = 0.7

G_thr = nx.Graph((u, v, d) for u, v, d in G.edges(data=True) if d.get("weight", 0.0) >= THR)
components = [c for c in nx.connected_components(G_thr) if len(c) > 1]
components = sorted(components, key=len, reverse=True)

print("n_components:", len(components))
print("top sizes:", [len(c) for c in components])

n_components: 35
top sizes: [709, 48, 24, 14, 5, 5, 4, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [105]:
def nunique_nonempty(x):
    x = pd.Series(x).dropna()
    x = x[x != ""]
    return x.nunique()

def is_consistent(group_df):
    return (
        nunique_nonempty(group_df["atcc_id"]) <= 1 and
        nunique_nonempty(group_df["media"])   <= 1
    )

In [106]:
consistent_groups = []
inconsistent_groups = []

for comp in components:
    sub = df[df["_node"].isin(comp)].copy()
    if is_consistent(sub):
        consistent_groups.append(sub)
    else:
        inconsistent_groups.append(sub)

print("consistent groups:", len(consistent_groups))
print("inconsistent groups:", len(inconsistent_groups))
print("top consistent sizes:", sorted([len(g) for g in consistent_groups], reverse=True)[:10])

consistent groups: 29
inconsistent groups: 6
top consistent sizes: [24, 14, 5, 5, 4, 4, 3, 3, 3, 2]


In [ ]:
# pick the biggest consistent group
g = consistent_groups[0]
assay_ids = g["assay_id"].unique()
print("group size:", len(g), "assays:", len(assay_ids))
print("ATCC non-empty uniques:", g.loc[g["atcc_id"]!="","atcc_id"].nunique())
print("media non-empty uniques:", g.loc[g["media"]!="","media"].nunique())


group size: 709 assays: 709
ATCC non-empty uniques: 8
media non-empty uniques: 47


In [108]:
g

,assay_id,assay_type,assay_organism,target_organism,organism_curated,doc_chembl_id,target_type,target_type_curated,target_type_curated_extra,target_chembl_id,target_chembl_id_curated,target_name_curated,bao_label,source_label,strain,atcc_id,mutations,known_drug_resistances,media,activity_type,unit,activities,nan_values,cpds,frac_cs,direction,act_flag,inact_flag,equal,higher,lower,dataset_type,cutoffs,ratios,cpds_qt,pos_ql,ratio_ql,cpds_ql,min_,p1,p25,p50,p75,p99,max_,Accepted,Considered,_node
1678,CHEMBL3865750,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3864192,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,assay format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,17,0,17,0.00012,-1.0,0,0,0,0,17,quantitative,5.0;10.0;20.0,0.941;1.0;1.0,17.0,NaN,NaN,NaN,3.220,3.257,3.636,3.866,4.320,5.229,5.314,NaN,NaN,"(CHEMBL3865750, MIC, umol.L-1)"
2064,CHEMBL5108516,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5107950,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,14,0,14,0.00010,-1.0,0,0,0,0,14,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,14.0,NaN,NaN,NaN,2.598,2.632,2.912,3.165,3.355,3.639,3.661,NaN,NaN,"(CHEMBL5108516, MIC, umol.L-1)"
3578,CHEMBL750921,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL1134115,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,6,0,6,0.00004,-1.0,0,0,6,0,0,quantitative,5.0;10.0;20.0,0.167;0.333;0.333,6.0,NaN,NaN,NaN,0.152,0.400,9.857,29.939,37.632,40.548,40.669,NaN,NaN,"(CHEMBL750921, MIC, umol.L-1)"
4189,CHEMBL4310743,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL4308875,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,5,0,5,0.00004,-1.0,0,0,4,0,1,quantitative,5.0;10.0;20.0,0.6;1.0;1.0,5.0,NaN,NaN,NaN,0.024,0.044,0.523,3.573,5.662,7.170,7.233,NaN,NaN,"(CHEMBL4310743, MIC, umol.L-1)"
4569,CHEMBL5347212,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL5346863,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,4,0,4,0.00003,-1.0,0,0,2,0,2,quantitative,5.0;10.0;20.0,0.5;1.0;1.0,4.0,NaN,NaN,NaN,3.261,3.280,3.723,6.045,8.213,8.213,8.213,NaN,NaN,"(CHEMBL5347212, MIC, umol.L-1)"
7344,CHEMBL715910,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL1135898,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,organism-based format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,1,0,1,0.00001,-1.0,0,0,1,0,0,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,1.0,NaN,NaN,NaN,2.510,2.510,2.510,2.510,2.510,2.510,2.510,NaN,NaN,"(CHEMBL715910, MIC, umol.L-1)"
7398,CHEMBL3865763,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3864192,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,assay format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,1,0,1,0.00001,-1.0,0,0,0,0,1,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,1.0,NaN,NaN,NaN,3.636,3.636,3.636,3.636,3.636,3.636,3.636,NaN,NaN,"(CHEMBL3865763, MIC, umol.L-1)"
7399,CHEMBL3865762,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3864192,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,assay format,LITERATURE,h37rv,,,KANAMYCIN,,MIC,umol.L-1,1,0,1,0.00001,-1.0,0,0,0,0,1,quantitative,5.0;10.0;20.0,1.0;1.0;1.0,1.0,NaN,NaN,NaN,3.819,3.819,3.819,3.819,3.819,3.819,3.819,NaN,NaN,"(CHEMBL3865762, MIC, umol.L-1)"
7400,CHEMBL3865761,F,Mycobacterium tuberculosis,Mycobacterium tuberculosis,Mycobacterium tuberculosis,CHEMBL3864192,ORGANISM,ORGANISM,ORGANISM,CHEMBL360,NaN,Mycobacterium tuberculosis,a